# Segmenting and Clustering Neighborhoods in Toronto : Assignment

# Section 1: Scraping the web page & Pre-processing

### Install and Import the necessary libraries 

Uncooment the below cell and **Install packages if not done already**

In [462]:
#!conda install -c conda-forge geopy --yes 
#!conda install -c conda-forge folium=0.5.0 

Then, Import the libraries needed. 

I have commented out the "display.max" option for ease of readability. If you need to see the full data frame, you can uncomment it.

In [463]:
import pandas as pd
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

import numpy as np

import urllib.request
import requests
#from bs4 import BeautifulSoup
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#Geocoder to convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
import folium 

print("Importing libraries complete!")

Importing libraries complete!


Below is the URL that contains the postal codes of Toronto (from Wikipedia)

In [464]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

### Scraping the web page 

**I am usinng the simplest way to read the table in to a Pandas Dataframe using the <code> read_html()</code> method** <br>
scaping the page and directly getting the tables in to a list using pandas "read_html()" <br>
There are three tables in this web page but we are interested in the first table. i.e <code> table_df[0] </code>

In [465]:
table_list = pd.read_html(url)
len(table_list)
table_df = table_list[0] # what we want is the first table in the list

table_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Pre processing the data frame

It seems that the Wikipedia page has already grouped the neighborhoods according to the postal codes. But we will do the below steps anyway.<br> 
First line of code drops the "Not assigned" cells from the dataframe <br>
Second line of code aggregates any multiple rows containing different neighborhood names but has the same postal code <br>
next two lines assigns the borough name to the neighborhood name, if the neighborhood name is empty 

In [466]:
Valid_df = table_df[table_df['Borough']!= "Not assigned"].reset_index(drop=True)

Toronto_df = Valid_df.groupby("Postal Code", as_index = False).agg(lambda x:', '.join(set(x)))

mask = Toronto_df['Neighborhood'] == "Not assigned"
Toronto_df.loc[mask, 'Neighborhood'] = Toronto_df.loc[mask, 'Borough']

print("Toronto_df.shape", Toronto_df.shape)
#Toronto_df.head(10)

Toronto_df.shape (103, 3)


# Section 2: Geo Coding - finding out the latitude & longitude

I'm usinng the csv file given in the below link to get the coordianntes of each of the postal codes.

In [467]:
Filepath = "http://cocl.us/Geospatial_data"
Coord_df = pd.read_csv(Filepath)

In [468]:
print(Coord_df.shape)
Coord_df.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Using the <code>pd.merge()</code> method to incorporate the coordinates in to the dataframe. Using an **inner join** o nthe **Postal Code** key

In [469]:
Neighbor_df = pd.merge(Toronto_df, Coord_df, how = 'inner', on = "Postal Code")
Neighbor_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Ploting the neighborhoods (Actually the postal codes)
<p>I'm using the location of "Casa Loma" as the center point for my map. Think it would look better than selecting Toronto.

In [470]:
address = 'Casa Loma, Toronto, CA'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6781015, -79.409415775.


In [471]:
# create map of Toronto using latitude and longitude values created above
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(Neighbor_df['Latitude'], Neighbor_df['Longitude'], Neighbor_df['Borough'], Neighbor_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)
    
map_Toronto

### Using the Foursquare API to explore venues arond our postal codes
<p> The url format for the API call is <br> <code> https://api.foursquare.com/v2/venues/explore?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&ll=LATITUDE,LONGITUDE&v=VERSION&limit=LIMIT </code>

In [472]:
## Defining Foursquare Credentials 
CLIENT_ID = 'HH15X12VLCIU2RJJ0233AP50Y4ZZ2ED1FNG5AHVAAK1FNUXB' # your Foursquare ID
CLIENT_SECRET = '0U2TM5EKCMVP1S11YS0XJBHNW5H2BMKMFDSVAVYAY0FQMHSD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#https://api.foursquare.com/v2/venues/explore?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&ll=LATITUDE,LONGITUDE&v=VERSION&limit=LIMIT

# Define the radius and limit of reponse from foursquare 
RADIUS = 500
LIMIT = 100

The below function is to get hte nearby venue details from the foursquare API, for every Postcode we have.

In [473]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Using the above function, create a new dataframe that contains details about all nearby venues. <br>
It seems that 2125 venues have been returned by the API. <br>
We can get more locaiton by changing the <code>LIMIT</code> and <code>RADIUS</code> parameters.

In [474]:
Toronto_venues = getNearbyVenues(names= Neighbor_df['Neighborhood'],
                                 latitudes= Neighbor_df['Latitude'],
                                 longitudes= Neighbor_df['Longitude'],
                                 radius = RADIUS,
                                 limit = LIMIT
                                )


print(Toronto_venues.shape)
Toronto_venues.head(7)

(2125, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern, Rouge",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
5,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
6,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


Grouping by Neighborhood will be improve readability. <p> And se see that there are 260 unique categories of locations 

In [475]:

Toronto_venues.groupby('Neighborhood').count()
Toronto_venues.head(10)
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 260 uniques categories.


Using one hot encoding

In [476]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
#Toronto_onehot
# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
#Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()
#Toronto_onehot

,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hospital,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Grouping the encoded dataframe by "Neighborhood"

In [477]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head(10)

Toronto_grouped.shape 

(95, 260)


If you wish to see the frequency of venues (or what type of venues appear most) in each of those neighborhoods, you can use the below code

<code>
num_top_venues = 5
for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
</code>


For the purpose of this assignment I'm focussing only on the top 10 venus categories for each neighborhood. 
<p> Below function helps to get the most common venue

In [478]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Usinng the function to see the most common venues

In [479]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Skating Rink,Dance Studio,Coffee Shop,Pub,Sandwich Place,Gym,American Restaurant,Deli / Bodega
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Sandwich Place,Sushi Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Supermarket,Ice Cream Shop,Deli / Bodega,Fried Chicken Joint
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Dog Run,Diner,Discount Store,Distribution Center,Doner Restaurant,Dessert Shop
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Thai Restaurant,Restaurant,Pizza Place,Butcher,Sushi Restaurant,Indian Restaurant,Pub


# Section 3: Clustering the Neighborhoods

Using kMeans clustering with mostly default parameters. 
<p> I tried many different values for the <code>n_clusters</code> value but finally decided to settle on 3 as it gave reasaonably explainable clusters.

In [480]:
# set number of clusters
kclusters = 3 

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(algorithm='auto', n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2,
       2, 2, 0, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2, 0,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2,
       0, 2, 2, 2, 2, 2, 0], dtype=int32)

In [481]:
#Just checking up how many neighborhoods are in each cluster 
clusters, counts = np.unique(kmeans.labels_, return_counts=True)

print("Total Neighborhoods: ", len(kmeans.labels_))
print(clusters, "\n", counts)

Total Neighborhoods:  95
[0 1 2] 
 [10  2 83]


## Preparing the Results dataframe with the cluster lables 

In [482]:

Toronto_result = Neighbor_df.drop(['Postal Code'], axis=1)
#Toronto_result.head()


# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_result = Toronto_result.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#Toronto_result.head() # check them visually 

Toronto_result = Toronto_result.dropna(axis='rows')
Toronto_result['Cluster Labels'] = Toronto_result['Cluster Labels'].astype(int)
Toronto_result.head(10)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2,Fast Food Restaurant,Print Shop,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2,Golf Course,Bar,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2,Electronics Store,Medical Center,Bank,Restaurant,Rental Car Location,Mexican Restaurant,Intersection,Breakfast Spot,Doner Restaurant,Dog Run
3,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Korean Restaurant,Convenience Store,Insurance Office,Electronics Store,Eastern European Restaurant,Drugstore,Ethiopian Restaurant,Donut Shop,Department Store
4,Scarborough,Cedarbrae,43.773136,-79.239476,2,Hakka Restaurant,Gas Station,Fried Chicken Joint,Bank,Bakery,Thai Restaurant,Athletics & Sports,Caribbean Restaurant,Lounge,Creperie
5,Scarborough,Scarborough Village,43.744734,-79.239476,2,Spa,Playground,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
6,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,2,Bus Station,Coffee Shop,Discount Store,Chinese Restaurant,Hobby Shop,Department Store,Yoga Studio,Distribution Center,Dim Sum Restaurant,Diner
7,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,2,Bakery,Park,Metro Station,Ice Cream Shop,Intersection,Bus Station,Bus Line,Soccer Field,Dog Run,Dim Sum Restaurant
8,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,2,Motel,Movie Theater,American Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Deli / Bodega,Dog Run
9,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,2,College Stadium,General Entertainment,Skating Rink,Café,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


## Creating the Map with (all) clustered neighborhoods 

Still using the previously generated center point for the map

In [483]:
#Using the previouslty created center point "Casa Loma"
print(latitude)
print(longitude)

43.6781015
-79.409415775


In [484]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters*2)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_result['Latitude'], Toronto_result['Longitude'], Toronto_result['Neighborhood'], Toronto_result['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining each cluster

### Cluster 1

The dataframe for cluster 1 shows that it is mostly "parks". 
<P> So if you love nature a lot these neighboorhods would be the best for you!. See the below map.

In [485]:
#Cluster 0 
Toronto_result.loc[Toronto_result['Cluster Labels'] == 0, Toronto_result.columns[[1] + list(range(5, Toronto_result.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Milliken, Agincourt North, Steeles East, L'Amo...",Playground,Park,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
23,York Mills West,Convenience Store,Electronics Store,Park,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Yoga Studio
25,Parkwoods,Food & Drink Shop,Park,Yoga Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
40,"East Toronto, Broadview North (Old East York)",Park,Pizza Place,Convenience Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
44,Lawrence Park,Park,Bus Line,Swim School,Yoga Studio,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
48,"Moore Park, Summerhill East",Tennis Court,Park,Playground,Trail,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
50,Rosedale,Park,Playground,Trail,Yoga Studio,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
74,Caledonia-Fairbanks,Park,Women's Store,Pool,Yoga Studio,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
90,"The Kingsway, Montgomery Road, Old Mill North",Park,River,Yoga Studio,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
98,Weston,Park,Yoga Studio,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


In [486]:
# generating a map for cluster 1
Cluster_1 = Toronto_result[Toronto_result['Cluster Labels'] == 0]

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to the map
for lat, lon, poi, cluster in zip(Cluster_1['Latitude'], Cluster_1['Longitude'], Cluster_1['Neighborhood'], Cluster_1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color= 'red',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 2

The most common venue for cluster2 is "Baseball field". 
<P> We can categorise these areas as baseballfield areas. See the below map.

In [487]:
#Cluster 1
Toronto_result.loc[Toronto_result['Cluster Labels'] == 1, Toronto_result.columns[[1] + list(range(5, Toronto_result.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,"Old Mill South, King's Mill Park, Sunnylea, Hu...",Baseball Field,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant
97,"Humberlea, Emery",Baseball Field,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant


In [488]:
# generating a map for cluster 2
Cluster_2 = Toronto_result[Toronto_result['Cluster Labels'] == 1]

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to the map
for lat, lon, poi, cluster in zip(Cluster_2['Latitude'], Cluster_2['Longitude'], Cluster_2['Neighborhood'], Cluster_2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color= 'green',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 3

The dataframe for cluster 2 shows that it is mostly "shops", "restuarants", "cafe" and other ammenities. 
<P> We can categorise these areas as mostly residential areas. See the below map.

In [489]:
#Cluster 2 
Toronto_result.loc[Toronto_result['Cluster Labels'] == 2, Toronto_result.columns[[1] + list(range(5, Toronto_result.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Malvern, Rouge",Fast Food Restaurant,Print Shop,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop
1,"Rouge Hill, Port Union, Highland Creek",Golf Course,Bar,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
2,"Guildwood, Morningside, West Hill",Electronics Store,Medical Center,Bank,Restaurant,Rental Car Location,Mexican Restaurant,Intersection,Breakfast Spot,Doner Restaurant,Dog Run
3,Woburn,Coffee Shop,Korean Restaurant,Convenience Store,Insurance Office,Electronics Store,Eastern European Restaurant,Drugstore,Ethiopian Restaurant,Donut Shop,Department Store
4,Cedarbrae,Hakka Restaurant,Gas Station,Fried Chicken Joint,Bank,Bakery,Thai Restaurant,Athletics & Sports,Caribbean Restaurant,Lounge,Creperie
5,Scarborough Village,Spa,Playground,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
6,"Kennedy Park, Ionview, East Birchmount Park",Bus Station,Coffee Shop,Discount Store,Chinese Restaurant,Hobby Shop,Department Store,Yoga Studio,Distribution Center,Dim Sum Restaurant,Diner
7,"Golden Mile, Clairlea, Oakridge",Bakery,Park,Metro Station,Ice Cream Shop,Intersection,Bus Station,Bus Line,Soccer Field,Dog Run,Dim Sum Restaurant
8,"Cliffside, Cliffcrest, Scarborough Village West",Motel,Movie Theater,American Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Deli / Bodega,Dog Run
9,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


In [490]:
# generating a map for cluster 3
Cluster_3= Toronto_result[Toronto_result['Cluster Labels'] == 2]

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to the map
for lat, lon, poi, cluster in zip(Cluster_3['Latitude'], Cluster_3['Longitude'], Cluster_3['Neighborhood'], Cluster_3['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color= 'blue',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters